這章討論  
* 單元測試與測試驅動開發的重要性
* 標準的 unittest 模組
* py.test 自動化測試套件
* mock 模組
* 程式範圍
* 以 tox 跨平台測試

# 為何要測試
---
撰寫測試有四個主要原因  
* 確保程式以開發者所想的方式運作
* 確保程式在修改後還能運作
* 確保開發者理解需求
* 確保程式以可維護的介面撰寫  
  
為了第一點撰寫程式所花的時間不能說明什麼  
我們可以直接在直譯器上直接測試  
當我們必須多次執行相同的測試動作時  
將測試自動化所花的時間比每次執行他們的時間還少

## 測試導向開發
兩個目標  
首先確保真正寫出測試  
其次  
先撰寫測試可讓我們專注於思考程式如何互動  
他可以告訴我們物件必須要有什麼方法與屬性如何存取  
幫助我們拆解問題  
然後透過測試過程式重組成較大且測試過的方案  

# 單元測試
---
從 Python 內建的測試函式庫開始探索  
單元測試專注於在任何一個測試內盡可能的測試最少量的程式  
此套件稱為 unittest  
他提供了幾個建構與執行單元測試的工具  
其中最重要的是 TestCase 類別  
此類別提供一組方法讓我們能夠比較值、設置測試和結束時進行清理  
當我們想要撰寫特定任務的單元測試時  
建構 TestCase 的子類別並撰寫執行實際測試的個別方法  
這些方法名稱必須以 test 開頭
測試會在過程中自動執行  

```
import unittest

class CheckNumbers(unittest.TestCase):
    def test_int_float(self) -> None:
        self.assertEqual(1, 1.0)
if __name__ == "__main__":
    unittest.main()
```
此程式只是製作 TestCase 的子類別並呼叫 TestCase.assertEqual方法  
此方法會根據兩個參數是否相等  
結果如下

In [12]:
! python unittest_sample1.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


加入字串與浮點數比較
```
import unittest

class CheckNumbers(unittest.TestCase):
    def test_int_float(self) -> None:
        self.assertEqual(1, 1.0)
        
    def test_str_float(self) -> None:
        self.assertEqual("1", 1.0)
        
if __name__ == "__main__":
    unittest.main()
```

In [14]:
! python unittest_sample2.py

.F
FAIL: test_str_float (__main__.CheckNumbers)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "unittest_sample2.py", line 8, in test_str_float
    self.assertEqual("1", 1.0)
AssertionError: '1' != 1.0

----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (failures=1)


結果第一行 .F 的部分  
第一個 . 代表第一個測試通過
後面的 F 代表第二個測試失敗  
並輸出一些資訊告訴我們何處失敗以及失敗數量總和  
TestCase中可以有任何數量的測試方法  
只要名稱以 test 開頭即可  
測試程序會執行每個獨立的測試  
每個測試應完全獨立於其他測試  
撰寫好的單元測試在於盡可能保持測試方法簡短

## 評測方法


assertRaises方法可用來確保特定函式呼叫會拋出特定例外  
也可以用 `with` 來包裝行內程式  
如果 `with` 陳述內的程式拋出特定例外則測試通過  
不然就失敗
以下為範例  
```
import unittest

def average(seq):
    return sum(seq)/len(seq)

class TestAverage(unittest.TestCase):
    def test_zero(self):
        self.assertRaises(ZeroDivisionError, average, [])

    def test_with_zero(self):
        with self.assertRaises(ZeroDivisionError):
            average([])

if __name__ == "__main__":
    unittest.main()
```

In [1]:
! python unittest_sample3.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


其他評測方法  
  
| 方法 | 說明 |
| --- | --- |
| assertGreater<br />assertGreaterEqual<br />assertLess<br />asserLessEqual | 接受兩個可比較的物件並確保與名稱所示比較相符 |
| assertIn<br />assertNotIn | 確保元素是(或不是)容器物件中的元素 |
| assertIsNone<br />assertIsNotNone | 確保元素是(或不是)None值 |
| assertSameElements | 確保兩個容器物件有相同元素，不看順序 |
| assertSequenceEqual<br />assertDictEqual<br />assertSetEqual<br />assertListEqual<br />asserTupleEqual | 確保兩個容易物件優相同元素與相同順序<br />如果失敗，顯示兩個清單的差異<br />後四個方法會測試清單的型別 |


## 減少模板與清理
撰寫過幾個小測試後  
通常會發現必須對幾個相關的測試執行同樣的設置程式  
舉例來說  
以下的 list 子類別有三個靜態計算方法

In [2]:
from collections import defaultdict

class StatsList(list):
    def mean(self):
        return sum(self)/len(self)

    def median(self):
        if len(self) % 2:
            return self[int(len(self)/2)]
        else:
            idx = int(len(self)/2)
            return (self[idx]+self[idx-1])/2
    
    def mode(self):
        freqs = defaultdict(int)
        for item in self:
            freqs[item] += 1
        mode_freq = max(freqs.values())
        modes = []
        for item, value in freqs.items():
            if value == mode_freq:
                modes.append(item)
        return modes

# 以上程式寫在 stats.py中

我想要對上述三個方法所做的測試狀況有非常類似的輸入  
想要看到遇到空清單或帶有非數值的清單  
或帶有正常資料時會發生什麼事  
我們可以使用 `TestCase` 類別的 `setUp` 方法來為每個測試執行初始化  
```
from stats import StatsList
import unittest

class TestValidInputs(unittest.TestCase):
    def setUp(self) -> None:
        '''測試初始化'''
        self.stats = StatsList([1,2,2,3,3,4])
    
    def test_mean(self):
        '''測試平均值是否等於 2.5'''
        self.assertEqual(self.stats.mean(), 2.5)
    
    def test_median(self):
        '''測試中位數是否等於 2.5 增加 4 之後是否等於 3'''
        self.assertEqual(self.stats.median(), 2.5)
        self.stats.append(4)
        self.assertEqual(self.stats.median(), 3)
    
    def test_mode(self):
        '''測試眾數是否為 [2,3], 移除一個2後, 眾數是否為 [3]'''
        self.assertEqual(self.stats.mode(), [2,3])
        self.stats.remove(2)
        self.assertEqual(self.stats.mode(), [3])

if __name__ == "__main__":
    unittest.main()
```

In [3]:
! python unittest_sample4.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


三個方法全部測試通過  
可以發現每次在測試方法時  
程式都會自動呼叫 `setUp`  
確保測試類別是以乾淨的狀態啟動  
例如測試 median 時增加了一個 4 進去  
卻不影響後面測試 mode

## 組織與執行測試
單元測試很快就會擴展開來  
要一次載入並測完全部會變得很複雜  
python 的 discover 模組可根據當前目錄與子目錄中查詢是否有任何 py 檔案名稱以 test 開頭  
如果這些檔案內有 TestCase 物件  
該測試會被執行
確保檔案名稱命名為 test_\<something\>.py  
然後執行 `python -m unittest discover`

## 忽略測試失敗
有時候已知測試會失敗  
但不想要程式報告此失敗  
Python 提供幾個裝飾器來標示測試預期會失敗或在已知條件下略過  
裝飾器包括:  
* expectedFailure()  失敗時不要紀錄
* skip(reason)  不執行測試，並描述為何要略過測試
* skipIf(condition, reason)  
* skipUnless(condition, reason) 
    
```
import unittest
import sys

class SkipTests(unittest.TestCase):
    @unittest.expectedFailure
    def test_fails(self):
        self.assertEqual(False, True)
    
    @unittest.skip('Test is useless')
    def test_skip(self):
        self.assertEqual(False, True)
    
    @unittest.skipIf(sys.version_info.minor == 4, 'broken on 3.7')
    def test_skipif(self):
        self.assertEqual(False, True)
    
    @unittest.skipUnless(sys.platform.startswith('linux'), 'broken unless on linux')
    def test_skipunless(self):
        self.assertEqual(False, True)

if __name__ == '__main__':
    unittest.main()
```

In [4]:
! python unittest_sample5.py

xsFs
FAIL: test_skipif (__main__.SkipTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "unittest_sample5.py", line 15, in test_skipif
    self.assertEqual(False, True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 4 tests in 0.000s

FAILED (failures=1, skipped=2, expected failures=1)


第一個 x 回報失敗，但是是預期中的失敗  
第二個不執行  
第三個 我使用的 python 為 3.7， 7 != 4， 因此跳過的條件不成立，執行後回報測試失敗  
第四個我的機器不為linux所以跳過

# py.test測試
---
py.test 與 unittest 有不同的布局  
他不需要將測試案例做成類別  
相反的  
他利用 Python 函式是物件的特性  
讓任何是當命名的函式行為如同測試  
相較於提供一堆自訂方法來測評  
他使用 assert 陳述來驗證節果  
這讓測試更容易閱讀和維護  
執行 py.test 時
會在當前目錄啟動並搜尋在該目錄或子目錄下任何以 test_ 字元開頭的 py 檔  
如果此檔案中的任何函式名稱也以 test 開頭  
他們會被當作個別測試來執行  
如果檔案中有任何名稱以 Test 開頭的類別  
該類別中 test_ 字元開頭的方法也會被執行  

In [1]:
! pip install pytest

You should consider upgrading via the 'C:\Users\user\pythonenv\python3_oop\Scripts\python.exe -m pip install --upgrade pip' command.


test_pytest1.py
  
```
def test_int_float():
    assert 1 == 1.0
```
  
test_pytest2.py
  
```
class TestNumbers:
    def test_int_float(self):
        assert 1 == 1.0
    def test_int_str(self):
        assert 1 == '1'
```

In [3]:
! py.test

============================= test session starts =============================
platform win32 -- Python 3.7.9, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: d:\94. Python\Python3OOP_Second_Edition\Ch12 測試物件導向程式
collected 3 items

test_pytest1.py .                                                        [ 33%]
test_pytest2.py .F                                                       [100%]

================================== FAILURES ===================================
__________________________ TestNumbers.test_int_str ___________________________

self = <test_pytest2.TestNumbers object at 0x0000028312345188>

    def test_int_str(self):
>       assert 1 == '1'
E       AssertionError: assert 1 == '1'

test_pytest2.py:5: AssertionError
=========================== short test summary info ===========================
FAILED test_pytest2.py::TestNumbers::test_int_str - AssertionError: assert 1 ...
========================= 1 failed, 2 passed in 1.05s =========================


可以發現  
第一個 test_pytest1.py 測試通過  
第二個 test_pytest2.py 一個通過一個沒過

## 執行設置與清理的一種方式
py.test支援類似 unittest 的設置與分解方法  
但前者更具彈性  
如果我們撰寫以類別為基礎的測試  
可以使用稱為 setup_method 與 teardown_method 方法  
他們在類別中測試方法前後呼叫以執行設置與清理任務  
與 unittest 一點不同的是  
這兩個方法都接受一個參數：代表被呼叫方法的函式物件  
  
此外  
py.test 提供其他設置與分解方法函式來讓我們更能控制設置與清理程式的時機  
setup_class 與 teardown_class 方法預期為類別方法  
接受要處理的類別的單一參數(不含self)  
  
最後  
我們有 setup_module 與 teardown_module  
他們在該模組測試前後執行  
這些〝一次性〞的設置很有用  
像是建構會用於模組中所有測試 socket 或資料庫連結等等  
要小心這個  
如果物件儲存了狀態  
會意外的在測試間引入相依性  

```
def setup_module(module):
    print(f"setting up MODULE {module.__name__}")

def teardown_module(module):
    print(f"tearing down MODULE {module.__name__}")

def test_a_function():
    print("Running Test Function")

class BaseTest:
    def setup_class(cls):
        print(f"setting up CLASS {cls.__name__}")
    
    def teardown_class(cls):
        print(f"tearing down CLASS {cls.__name__}\n")
    
    def setup_method(self, method):
        print(f"setting up METHOD {method.__name__}")
    
    def teardown_method(self, method):
        print(f"tearing down METHOD {method.__name__}")

class TestClass1(BaseTest):
    def test_method_1(self):
        print("Running METHOD 1-1")
    
    def test_method_2(self):
        print("Running METHOD 1-2")

class TestClass2(BaseTest):
    def test_method_1(self):
        print("Running METHOD 2-1")
    
    def test_method_2(self):
        print("Running METHOD 2-2")
```

BaseTest類別唯一目的是擷取四個方法  
並使用繼承減少重複的程式碼  
從 py.test 角度來看  
這兩個子類別不只兩個測試方法  
還有兩個設置與兩個分解(一個在類別階層、一個在方法階層)

In [5]:
! py.test setup_teardown.py -s

============================= test session starts =============================
platform win32 -- Python 3.7.9, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: d:\94. Python\Python3OOP_Second_Edition\Ch12 測試物件導向程式
collected 5 items

setup_teardown.py 
setting up MODULE setup_teardown
Running Test Function
.setting up CLASS TestClass1
setting up METHOD test_method_1
Running METHOD 1-1
.tearing down METHOD test_method_1
setting up METHOD test_method_2
Running METHOD 1-2
.tearing down METHOD test_method_2
tearing down CLASS TestClass1

setting up CLASS TestClass2
setting up METHOD test_method_1
Running METHOD 2-1
.tearing down METHOD test_method_1
setting up METHOD test_method_2
Running METHOD 2-2
.tearing down METHOD test_method_2
tearing down CLASS TestClass2

tearing down MODULE setup_teardown


============================== 5 passed in 0.06s ==============================


模組的設置與分解方法在階段前後執行  
然後模組層級的測試函式執行  
接下來  
第一個類別設置方法執行  
接著是該類別的兩個測試  
這些測試都包裝在 setup_method 與 teardown_method 呼叫中  
在所有測試執行後  
類別的分解方法被呼叫  
同樣的順序發生在第二類別  

## 完全不同的設置變數方式
